In [1]:
import requests
from pprint import pprint
import sqlite3

In [2]:
# API key you received from NREL
api_key = "RjZpx6DxeHBP4d17uK9Uifu6qhas3674psy6dJ7Q"



In [3]:
# Base URL for the NREL API
base_url = "https://developer.nrel.gov"
endpoint = "/api/alt-fuel-stations/v1.json"

url = base_url + endpoint



In [4]:
# Set up parameters: Electric Stations in Texas
params = {
    'api_key': api_key,
    'limit': 10,  # Limiting the number of results returned
    'fuel_type': 'ELEC',  # For electric stations
    'state': 'TX'  # For stations located in Texas
}



In [5]:
conn = sqlite3.connect('fuel_stations.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS fuel_stations (
        id INTEGER PRIMARY KEY,
        station_name TEXT,
        address TEXT,
        fuel_type_code TEXT,
        state TEXT,
        latitude REAL,
        longitude REAL
    )
''')
conn.commit()

In [7]:
# Make a GET request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    json_data = response.json()

    # Insert each record into the table
    for station in json_data['fuel_stations']:
        cursor.execute('''
            INSERT INTO fuel_stations (id, station_name, address, fuel_type_code, state, latitude, longitude)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            station['id'],
            station['station_name'],
            station['street_address'],
            station['fuel_type_code'],
            station['state'],
            station['latitude'],
            station['longitude']
        ))

    # Commit the changes and close the connection
    conn.commit()
else:
    print(f"Failed to retrieve data: {response.status_code}")

In [8]:
# Execute a SELECT query to retrieve all records from the table
cursor.execute('SELECT * FROM fuel_stations')

# Fetch all the records
rows = cursor.fetchall()

# Print the retrieved data
for row in rows:
    print(row)

# Close the database connection
conn.close()

(36378, 'BROOKFIELD PROP 300 CLAY STAT1', '300 Clay St', 'ELEC', 'TX', 29.756604, -95.372854)
(39114, 'Clay Cooley Nissan', '4914 S IH-35', 'ELEC', 'TX', 30.206145, -97.759109)
(39708, 'McGavock Nissan - Abilene', '818 E Overland Trail', 'ELEC', 'TX', 32.408909, -99.771463)
(39709, 'McGavock Nissan - Amarillo', '4700 S Soncy Rd', 'ELEC', 'TX', 35.159354, -101.938937)
(39710, 'Don Davis Nissan', '1320 E Interstate 20', 'ELEC', 'TX', 32.676458, -97.087169)
(39711, 'Town North Nissan', '9160A Research Blvd', 'ELEC', 'TX', 30.374542, -97.731445)
(39712, 'Baytown Nissan', '5445 I-10 E', 'ELEC', 'TX', 29.80725299717069, -94.97383364868773)
(39713, 'Mike Smith Nissan', '1515 Interstate 10 S', 'ELEC', 'TX', 30.062535, -94.133511)
(39714, 'Grubbs Nissan', '310 Airport Fwy', 'ELEC', 'TX', 32.839339, -97.160994)
(39715, 'Douglass Nissan', '1001 Earl Rudder Fwy S', 'ELEC', 'TX', 30.639201, -96.30671)
